## Introduction

[Dask](https://dask.org/) is a Python library suited for block-wise parallelization of most Numpy and Pandas tasks. It is also useful for performing operations on out-of-memory data by automating the processing of paging chunks to and from storage. While there is a wonderful [tutorial](https://github.com/dask/dask-tutorial) available on GitHub, it does not show off some of the more interesting capabilities of Dask. The purpose of this notebook is to demonstrate how to use the `map_overlap` function to downscale an out-of-memory volumetric (3D) image. That being said, this notebook will make more sense if you go through the tutorial first.

### More Details

The way Dask operates is by creating a lazily-evaluated, directed acyclic graph of all the tasks its needs to perform to carry out its instructions, and assigning tasks in the correct order to workers in a distributed environment. What that means is that Dask doesn't do any computation until data is actually needed, or instructed directly by the `.compute()` method. The acyclic directed task graph part means that Dask figures out the order of operations necessary to perform what you've asked it to do and what operations depend on what other operations, and sets them up following the logic of the code you've written. Acyclic means that there can't be any cycles, which means no while loops and no recursion. When Dask is explicitly (via `.compute()`) or implicitly (via `.imshow()`, etc.) instructed to perform computations, the workers are assigned tasks from the task graph in the order indicated.

It is possible to visualize the task graph using `.visualize()`. The method is not used in this notebook because the task graphs are large and difficult to see in the drawing. The tutorial referenced above is a good source for more information. Progress of the tasks may be visualized on the task graph using the real-time dashboard, which will be explained closer to where it is set up.

## Setup

To use this notebook, you will need the following packages:
 - dask (>=2.5.2)
 - distributed (>=2.5.2)
 - numpy (>=1.16.5)
 - scikit-image (>=0.15.0)

Assuming you are using conda as your package manager (recommended!), cloning a new base enviroment using `conda create --name <name> --clone base`, activating the environment using either `conda activate <name>` or `source activate <name>` and running `conda install dask` should be sufficient.

### Library Imports and Helper Functions

Here we import packages and set up a couple of useful helper functions for downstream computations.

In [1]:
import dask
import dask.array as da
import numpy as np

In [2]:
def compute_mem_mb(shape):
    '''Determines memory consumption of an array with shape in MB'''
    from functools import reduce
    from operator import mul
    count = reduce(mul, shape, 1)
    return count * 8 / ( 1024 ** 2 )

def display(image):
    '''Shows an image in the Jupyter notebook.'''
    from skimage import io
    io.imshow(image)
    io.show()

def throughput(mem_mb, time_s):
    print("MB/s: {:.2f}".format(mem_mb / time_s))

def stats_da(image):
    print("Min: {min:.2f}\nMax: {max:.2f}\nMean: {mean:.2f}\nStd: {std:.2f}".format(
        max=image.max().compute(),
        min=image.min().compute(),
        mean=image.mean().compute(),
        std=image.std().compute()))

def stats_np(image):
    print("Min: {min:.2f}\nMax: {max:.2f}\nMean: {mean:.2f}\nStd: {std:.2f}".format(
        max=image.max(),
        min=image.min(),
        mean=image.mean(),
        std=image.std()))

def scale_shape(shape, factor):
    '''Gives the expected shape if scaled by factor'''
    return tuple(map(lambda x: round(x * factor), shape))

def rescale(chunk, factor, order=0):
    '''Rescales a chunk based on scaling factor.'''
    from skimage.transform import rescale
    return rescale(chunk, factor, order=order, multichannel=False)

def mean_downscale(chunk, factors):
    '''Resizes a chunk using the integer-factor local-mean downscaling from scikit-image'''
    from skimage.transform import downscale_local_mean
    return downscale_local_mean(chunk, factors)

### Distibuted Client and Dashboard Setup

As mentioned earler, Dask comes with a helpful real-time dashboard for understanding how tasks are distributed to workers. The dashboard is useful for developing intuition about parallel code performance. More information is available in the Dask tutorial, linked in the introduction at the top of the notebook. Below we set up the Dask distributed client and get a URL for the dashboard.


In [3]:
from dask.distributed import Client
c = Client()
port = c.scheduler_info()['services']['dashboard']
print("Type `http://localhost:{port}` into the URL bar of your favorite browser to watch the following code in action on your machine in real time.".format(port=port))

## Computation

### Preparation

Here we prepare some basic constants for downstream computation, like volumetric image size and chunk size. If at any point you receive any error messages about running out of memory on workers, reduce the chunk size. Feel free to play around with these values and re-run the notebook. It is recommended to follow along with the dashboard in your browser, see above for more information. Keep in mind that chunks can't be larger than the shape. Chunks that are too small will create significant overhead in the task graph and require more communication between workers, reducing efficiency. Chunks that are too large may consume too much memory.

In [4]:
ndim = 3
size_len = 400
size = ndim * [size_len]
mem_mb = compute_mem_mb(size)
print("3D array size (MB): {:.2f}".format(mem_mb))
chunk_len = 100
chunks = ndim * [chunk_len]
chunk_mem_mb = compute_mem_mb(chunks)
print("Chunk size (MB): {:.2f}".format(chunk_mem_mb))

Here we create a vaguely interesting array of values which increase radially from the origin. To do this, we use the `da.indices()` function to create a collection of Dask arrays whose values are equal to their index. One array is created per dimension, giving us three arrays in the volumetric case.

We then stack the three arrays using `da.stack()` along the fourth dimension (`axis=3`) to create a 3D array of coordinate vectors.

To create the radial image we square every element, sum along the fourth dimension to add up the squares along the coordinate dimension (`axis=3` again), and then find the square root of every element.

Finally we normalize the array values by the maximum value over the entire array.

The construct `t = %timeit -n1 -r1 -o -q fn()` is IPython magic which means:
 - `%timeit`: time `fn()` on this line only...
 - `-n1`: looping one time...
 - `-r1`: repeating loops one time...
 - `-o`: output a value (and assign to `t`)...
 - `-q`: silently.

In [5]:
coords = da.indices(size, dtype='float', chunks=chunks)
grid = da.stack(coords, axis=3)
base_image = (grid ** 2).sum(axis=3) ** 0.5
image = base_image / base_image.max()
t = %timeit -n1 -r1 -o -q display(image[(None, 0)].squeeze())
throughput(mem_mb, t.average)

Here we generate some image statistics.

In [6]:
t = %timeit -n1 -r1 -o -q stats_da(image)
throughput(mem_mb, t.average)

### Image Rescaling

It is worth noting that any naive block-based image rescaling is likely to create artifacts somewhere in the image, typically at the image boundary or at the interface between blocks. Unfortunately a careful, distributed implementation of rescaling is not available at the time of writing, so we have to use our own naive implementation.

Here we perform image rescaling using the local-mean downscaling method available in `scikit-image`. This downscaling method uses an integer scaling `factor`, and returns the mean of values in a cube-shaped window whose side length is equal to factor. The window slides across intervals equal to `factor`. With the above note in mind, we should be careful that the chunk size is divisible by the integer scaling factor to ensure proper integer scaling.  by having the image size divisible by the scaling factor, but this is not always possible in practice.

An important detail is that we can't use `map_overlaps()` directly because the function being mapped (`mean_downscale`) changes the chunk size. The `map_overlaps()` method assumes the same depth for both the overlapping and trimming operations, which is not correct for rescaling. For our problem, an overlap equal to `factor` is reduced to a trim of 1.

Note that a larger factor reduces throughput. This is because the `overlap` functionality creates new chunks from the originals padded by values in neighboring chunks up to the supplied depth. If the chunk is at the image boundary, the supplied boundary is used there. In our case, a reflected boundary is used. In this particular problem, the issue doesn't become apparent until a large scaling factor is used. Try setting `factor` equal to the chunk length. On my machine the computation took about 5 times as long as it did for values equal to or below 20. The reason for this is the increase in inter-worker communication.

In [7]:
window_factor = 5
window_factors = ndim * tuple([window_factor])
boundary = 'reflect'

overlapped = da.overlap.overlap(image, depth=window_factor, boundary=boundary)

new_chunks = tuple(map(lambda x: round(x / window_factor), chunks))
mapped = overlapped.map_blocks(lambda x: mean_downscale(x, window_factors), chunks=new_chunks, dtype=image.dtype)

import time
start = time.time()

local_mean_scaled_image = da.overlap.trim_overlap(mapped, depth=1).compute()

end = time.time()
t = end - start

In [8]:
throughput(mem_mb, t)
display(local_mean_scaled_image[(None, 0)].squeeze())
stats_np(local_mean_scaled_image)
print("Output shape: {}".format(local_mean_scaled_image.shape))
print("Expected shape: {}".format(scale_shape(image.shape, 1 / window_factor)))

You may have noticed that because the chunks are divisble by the scaling factor, we don't need overlapping at all! To explain, consider that the overlap is equal to the factor. Because the window slides by a distance equal to factor and the overlap depth is equal to factor, the window never combines overlapped values with chunk values. Given this fact, we can rewrite the previous code more simply as below using only a call to `map_blocks()` without calls to `overlaps`. Note the maximum error is exactly zero, indicating no difference in code output.

In [9]:
import time
start = time.time()

local_mean_scaled_image_s = image.map_blocks(lambda x: mean_downscale(x, window_factors), dtype=image.dtype).compute()

end = time.time()
t = end - start
throughput(mem_mb, t)

error = np.abs(local_mean_scaled_image - local_mean_scaled_image_s)
display(error[(None, 0)].squeeze())

max_error = error.max()
identical = max_error == 0.0
print("Maximum error: {:e}".format(max_error))
print("Identically zero: {}".format(identical))

print("Output shape: {}".format(local_mean_scaled_image_s.shape))
print("Expected shape: {}".format(scale_shape(image.shape, 1 / window_factor)))

What happens if we replace `window_factor` with a value that doesn't evenly divide `chunk_len`? Try setting `window_factor = 7` in the previous cells and rerunning them. Notice anything off with the previous error image? There should be a grid-patterned artifact. The reason for the artifacting is because `window_factor` doesn't evenly divide `chunk_len`. Recall that the window slides by a value equal to `window_factor`, which means the windows at the ends of each chunk axis lie partly outside the chunk. Dask appears to use the constant value 0 for elements outside the chunk. As a result, the value of the mean is reduced in windows which lie partly out of a chunk, causing the grid artifact.

### Spline Interpolation Rescaling

Here we perform cubic spline interpolation scaling on the image. The value of `factor` plays a different role here. Rather than being an integer window size, it represents the factor to multiply the shape of the image. We are intentionally using a factor value which does not produce an even multiple of 

In [15]:
factor = 1 / 3

import time
start = time.time()

cubic_scaled_image = image.map_blocks(lambda x: rescale(x, factor, order=3), dtype=image.dtype).compute()

end = time.time()
t = end - start

In [16]:
throughput(mem_mb, t)
display(cubic_scaled_image[(None, 0)].squeeze())
stats_np(cubic_scaled_image)
print("Output shape: {}".format(cubic_scaled_image.shape))
print("Expected shape: {}".format(scale_shape(image.shape, factor)))

### Validation

To be sure the results align with expectation, we can rescale a Numpy array of a slice together with a Dask `map_blocks()` rescaling of the same slice, and then compare them. For this sort of validation to work we need a scaling factor that results in an integer shape to ensure the shape is the same for both rescaled images.

As we can see below, the 

In [18]:
factor = 1 / 4

image_slice_da = image[(None, 0)].squeeze()
cubic_slice_da = np.asarray(image_slice_da.map_blocks(lambda x: rescale(x, factor, order=3), dtype=image_slice_da.dtype).compute())

image_slice_np = np.asarray(image_slice_da.compute())
cubic_slice_np = rescale(image_slice_np, factor, order=3)

error_slice = np.abs(cubic_slice_da - cubic_slice_np)
max_error_slice = error_slice.max()
display(error[(None, 0)].squeeze())

identical = max_error == 0.0
print("Maximum error: {:e}".format(max_error))
print("Identically zero: {}".format(identical))

print("Output shape: {}".format(local_mean_scaled_image_s.shape))
print("Expected shape: {}".format(scale_shape(image.shape, 1 / window_factor)))

## Concluding Remarks

We hope you found this brief notebook useful for understanding how Dask can be applied to certain image analysis problems using `overlap` and `map_blocks`. If you need more information, please refer to the Dask website linked in the introduction. If you liked this notebook, feel free to contact the author.